In [1]:
from random import randint
from numpy import array
from numpy import argmax
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [2]:
# Limit GPU VRAM
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1700)]) 

In [3]:
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
    return [randint(0, 99) for _ in range(length)]

In [4]:
# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)

In [5]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [6]:
# generate data for the lstm
def generate_data():
    # generate sequence
    sequence = generate_sequence()
    # one hot encode
    encoded = one_hot_encode(sequence)
    # create lag inputs
    df = DataFrame(encoded)
    df = concat([df.shift(4), df.shift(3), df.shift(2), df.shift(1), df], axis=1)
    # remove non-viable rows
    values = df.values
    values = values[5:,:]
    # convert to 3d for input
    X = values.reshape(len(values), 5, 100)
    # drop last value from y
    y = encoded[4:-1,:]
    return X, y

In [7]:
# define model
model = Sequential()
model.add(LSTM(50, batch_input_shape=(5, 5, 100), stateful=True))
model.add(Dense(100, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
# fit model
for i in range(20):
    X, y = generate_data()
    model.fit(X, y, epochs=1000, batch_size=5, verbose=1, shuffle=False)
    model.reset_states()

Epoch 1/1000
4/4 [==============================] - 0s 4ms/step - loss: 5.3038 - acc: 0.0500
Epoch 2/1000
4/4 [==============================] - 0s 4ms/step - loss: 5.3376 - acc: 0.0500
Epoch 3/1000
4/4 [==============================] - 0s 4ms/step - loss: 4.4291 - acc: 0.1500
Epoch 4/1000
4/4 [==============================] - 0s 5ms/step - loss: 3.8470 - acc: 0.1000
Epoch 5/1000
4/4 [==============================] - 0s 5ms/step - loss: 3.2265 - acc: 0.1000
Epoch 6/1000
4/4 [==============================] - 0s 6ms/step - loss: 2.8710 - acc: 0.3000
Epoch 7/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.5046 - acc: 0.3500
Epoch 8/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.2894 - acc: 0.5500
Epoch 9/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.9974 - acc: 0.5000
Epoch 10/1000
4/4 [==============================] - 0s 5ms/step - loss: 1.5936 - acc: 0.7500
Epoch 11/1000
4/4 [==============================] - 0s 4ms/step - lo

In [13]:
# evaluate model on new data
X, y = generate_data()
yhat = model.predict(X, batch_size=5)
print('Expected:  %s' % one_hot_decode(y))
print('Predicted: %s' % one_hot_decode(yhat))

Expected:  [82, 24, 13, 99, 13, 54, 65, 56, 51, 43, 6, 66, 44, 25, 80, 5, 61, 8, 4, 34]
Predicted: [18, 47, 33, 18, 85, 31, 4, 36, 56, 56, 56, 5, 5, 34, 5, 61, 34, 56, 17, 18]
